In [1]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
   width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [5]:
import cv2

In [2]:
import os
# https://stackoverflow.com/a/9728478
def list_files(startpath):
    files_path = []
    for root, dirs, files in os.walk(startpath):
        for indx in range(len(files)):
            files_path.append(root+"/"+files[indx])
    return list(filter(lambda el: el.endswith("JPG"), files_path))

In [3]:
# Obtengo todas las imagenes que se descargaron
home_path = "/media/juan/jc/archivos_represion/Ficheros/"
imgs_paths = list_files(home_path)
print("{:,} imagenes encontradas dentro de la carpeta Ficheros".format(len(imgs_paths)))

18,008 imagenes encontradas dentro de la carpeta Ficheros


In [10]:
%%time
import multiprocessing as mp
print("Procesando con {} nucleos".format(mp.cpu_count()))
def get_shape_information(url):
    shp = cv2.imread(url).shape
    return (url, *shp, 1 if shp[1]>shp[0] else 0)
pool = mp.Pool(mp.cpu_count())
results = pool.map(get_shape_information, imgs_paths)
pool.close()


Procesando con 8 nucleos
CPU times: user 163 ms, sys: 43.2 ms, total: 207 ms
Wall time: 25min 13s


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName('explotacion').getOrCreate()

In [14]:
from pyspark.sql.types import *

cSchema = StructType([StructField("path", StringType()), StructField("h", IntegerType()),StructField("w", IntegerType()),StructField("c", IntegerType()), StructField("is_horizontal", IntegerType())])


df = spark.createDataFrame(results,schema=cSchema) 

In [15]:
%%time
df.toPandas().to_csv("data/img_features.csv", index=False)

CPU times: user 231 ms, sys: 7.79 ms, total: 239 ms
Wall time: 5.1 s


In [19]:
df.groupBy("w", "h").count().withColumn("portcentaje",((F.col("count")*100)/18008).alias("porcentaje")).orderBy(F.col("w"),F.col("h")).show(truncate=False)

+----+----+-----+-------------------+
|w   |h   |count|portcentaje        |
+----+----+-----+-------------------+
|1728|2304|792  |4.398045313194136  |
|1944|2592|6    |0.03331852509995557|
|2304|1728|1876 |10.417592181252777 |
|2448|3264|722  |4.009329187027988  |
|2592|1944|2637 |14.643491781430475 |
|3264|2448|413  |2.2934251443802753 |
|3456|4608|2662 |14.782318969346957 |
|4320|2432|188  |1.0439804531319414 |
|4608|3456|8712 |48.3784984451355   |
+----+----+-----+-------------------+



In [23]:
df.groupBy("is_horizontal").count().withColumn("portcentaje",((F.col("count")*100)/18008).alias("porcentaje")).orderBy(F.col("is_horizontal").desc()).show(truncate=False)

+-------------+-----+------------------+
|is_horizontal|count|portcentaje       |
+-------------+-----+------------------+
|1            |13826|76.77698800533096 |
|0            |4182 |23.223011994669037|
+-------------+-----+------------------+



In [24]:
df.show(2, truncate=False)

+-------------------------------------------------------------------------------------------+----+----+---+-------------+
|path                                                                                       |h   |w   |c  |is_horizontal|
+-------------------------------------------------------------------------------------------+----+----+---+-------------+
|/media/juan/jc/archivos_represion/Ficheros/ACNR_militantes/Adame_Flores_samuel/IMG_6837.JPG|3456|4608|3  |1            |
|/media/juan/jc/archivos_represion/Ficheros/ACNR_militantes/Adame_Flores_samuel/IMG_6838.JPG|3456|4608|3  |1            |
+-------------------------------------------------------------------------------------------+----+----+---+-------------+
only showing top 2 rows



In [28]:
count_udf = F.udf(lambda str: str.count("/"))
df.withColumn("levels", count_udf(F.regexp_replace(F.col("path"), "/media/juan/jc/archivos_represion/Ficheros/", ""))).groupBy(F.col("levels")).count().orderBy(F.col("levels")).show(truncate=False)

+------+-----+
|levels|count|
+------+-----+
|1     |3699 |
|2     |12797|
|3     |1512 |
+------+-----+



In [34]:
rpl_udf = F.udf(lambda pth: pth.replace(pth.split("/")[-1], ""))
mode_udf = F.udf(lambda arr: max(set(arr), key=arr.count))
df.withColumn("parent",rpl_udf(F.col("path"))).groupBy("parent").count().agg(
 F.avg("count").alias("avg_count"),
 F.min("count").alias("min_count"),
 F.max("count").alias("max_count"),
    F.collect_list(F.col("count")).alias("nums")
).withColumn("moda", mode_udf(F.col("nums"))).drop("nums").show(truncate=False)

+---------+---------+---------+----+
|avg_count|min_count|max_count|moda|
+---------+---------+---------+----+
|36.016   |1        |684      |1   |
+---------+---------+---------+----+



In [38]:
rpl_udf = F.udf(lambda pth: pth.replace(pth.split("/")[-1], ""))
mode_udf = F.udf(lambda arr: max(set(arr), key=arr.count))
df.withColumn("parent",rpl_udf(F.col("path"))).groupBy("parent").count().groupBy(F.col("count").alias("nm")).count().orderBy(F.col("nm").asc()).show(100, truncate=False)

+---+-----+
|nm |count|
+---+-----+
|1  |66   |
|2  |45   |
|3  |45   |
|4  |28   |
|5  |27   |
|6  |19   |
|7  |12   |
|8  |11   |
|9  |9    |
|10 |12   |
|11 |4    |
|12 |11   |
|13 |6    |
|14 |4    |
|15 |4    |
|16 |4    |
|17 |8    |
|18 |2    |
|19 |3    |
|20 |4    |
|21 |2    |
|22 |6    |
|23 |5    |
|24 |7    |
|25 |3    |
|26 |4    |
|27 |5    |
|28 |1    |
|29 |3    |
|30 |2    |
|31 |2    |
|32 |2    |
|33 |2    |
|34 |1    |
|36 |1    |
|37 |1    |
|38 |1    |
|39 |1    |
|40 |2    |
|43 |2    |
|44 |2    |
|45 |1    |
|47 |2    |
|49 |3    |
|52 |1    |
|54 |3    |
|55 |1    |
|57 |1    |
|58 |4    |
|59 |1    |
|60 |1    |
|61 |2    |
|62 |1    |
|66 |1    |
|68 |3    |
|69 |1    |
|70 |3    |
|71 |1    |
|72 |2    |
|74 |1    |
|77 |2    |
|78 |1    |
|79 |2    |
|80 |1    |
|81 |3    |
|82 |2    |
|83 |1    |
|85 |1    |
|86 |2    |
|90 |1    |
|92 |2    |
|95 |2    |
|96 |1    |
|97 |2    |
|100|1    |
|103|6    |
|105|2    |
|106|1    |
|107|1    |
|108|1    |
|112